【4時間でPython AIアプリ開発】Python Django scikit learnでAIアプリを開発  
https://www.youtube.com/watch?v=_87RY9rb79I&list=WL&index=15

# インポート

In [51]:
import pandas as pd
from collections import Counter
import string
import jctconv
import emoji
import re

# データチェック

In [4]:
!ls

all_rakuma.csv   rakuma_NPL.ipynb


In [5]:
df = pd.read_csv('all_rakuma.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122070 entries, 0 to 122069
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   date           122070 non-null  object
 1   item_url       122070 non-null  object
 2   title          122070 non-null  object
 3   price          122070 non-null  int64 
 4   brand          122070 non-null  object
 5   category       122070 non-null  object
 6   good           122070 non-null  int64 
 7   bad            122070 non-null  int64 
 8   normal         122070 non-null  int64 
 9   ship_days      122070 non-null  object
 10  comments       11186 non-null   object
 11  last_log_date  122070 non-null  object
 12  good_counts    122070 non-null  int64 
 13  prefecture     122070 non-null  object
 14  delivery_fee   122070 non-null  object
 15  seller_url     122070 non-null  object
 16  size           122070 non-null  object
 17  picture_url    122070 non-null  object
 18  desc

In [6]:
df.head()

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,comments,last_log_date,good_counts,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict
0,2019/10/08 23:39:51,https://item.fril.jp/055f4080a2a0ae275db6ba7be...,Epiphone LTD Korina Explorer Bass,40000,Epiphone,楽器 > ベース > エレキベース,109,0,0,1-2日後,NaN,2000-01-01,0,岐阜県,着払い,https://fril.jp/shop/42df208014145c438f96e01f5...,なし,https://img.fril.jp/img/259586636/l/734809353....,コリーナボディ ブラックピックガード ゴールドパーツの豪華なエクスプローラー コリーナ ...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', 'ブランド':..."
1,2019/10/08 23:39:51,https://item.fril.jp/896a1d0197d05cf8431c136d9...,TVB-100 (SUN),38000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,"['TOKYO GUITARS', '本モデルを含め、TOKYO GUITARS工房にて製造...",2019/08/31,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223532100/l/635334018....,Tokyo Guitars モデル名 TVB 100 SUN Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
2,2019/10/08 23:39:51,https://item.fril.jp/6bdb6b4dd0f6c518938b94ec2...,TRB-4000 (BL),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294381/l/634674588....,Tokyo Guitars モデル名 TRB 4000 BL Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
3,2019/10/08 23:39:51,https://item.fril.jp/b89c82258515163e5aa36aa48...,TRB-4000 (FG),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294160/l/634673947....,Tokyo Guitars モデル名 TRB 4000 FG Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
4,2019/10/08 23:39:51,https://item.fril.jp/cf904ef67c6cec09a7465e7f4...,TB-100 (BRWN),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/216145047/l/614817207....,Tokyo Guitars モデル名 TB 100 BRWN Tokyo Guit...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."


In [12]:
df.category.nunique()

1012

In [10]:
df.category.value_counts() / len(df) *100

メンズ > 靴/シューズ > スニーカー                 9.174244
エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ    8.975178
エンタメ/ホビー > フィギュア > アニメ/ゲーム           5.418203
エンタメ/ホビー > CD > ポップス/ロック(洋楽)         2.919636
エンタメ/ホビー > フィギュア > 特撮                2.577210
                                       ...   
エンタメ/ホビー > 雑誌 > 絵本/児童書               0.000819
スマホ/家電/カメラ > 冷暖房/空調 > その他            0.000819
メンズ > ジャケット/アウター > ライダースジャケット        0.000819
楽器 > DTM/DAW > DAWソフトウェア             0.000819
エンタメ/ホビー > 漫画 > 女性漫画                 0.000819
Name: category, Length: 1012, dtype: float64

In [14]:
ctg_df = df.category.value_counts().reset_index() #Seriesにresetindexをするとdfになる
ctg_df

,index,category
0,メンズ > 靴/シューズ > スニーカー,11199
1,エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ,10956
2,エンタメ/ホビー > フィギュア > アニメ/ゲーム,6614
3,エンタメ/ホビー > CD > ポップス/ロック(洋楽),3564
4,エンタメ/ホビー > フィギュア > 特撮,3146
...,...,...
1007,エンタメ/ホビー > 雑誌 > 絵本/児童書,1
1008,スマホ/家電/カメラ > 冷暖房/空調 > その他,1
1009,メンズ > ジャケット/アウター > ライダースジャケット,1
1010,楽器 > DTM/DAW > DAWソフトウェア,1


In [17]:
over1000_data = ctg_df[ctg_df.category > 1000]
over1000_data

,index,category
0,メンズ > 靴/シューズ > スニーカー,11199
1,エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ,10956
2,エンタメ/ホビー > フィギュア > アニメ/ゲーム,6614
3,エンタメ/ホビー > CD > ポップス/ロック(洋楽),3564
4,エンタメ/ホビー > フィギュア > 特撮,3146
5,エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー,2780
6,スポーツ/アウトドア > フィッシング > ルアー用品,2562
7,エンタメ/ホビー > トレーディングカード > シングルカード,2521
8,エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー,2162
9,エンタメ/ホビー > CD > アニメ,2020


In [18]:
over1000_data.category.sum()

63006

In [20]:
data = df.merge(over1000_data[['index']], left_on='category', right_on='index')
data

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,...,last_log_date,good_counts,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict,index
0,2019/10/09 15:12:05,https://item.fril.jp/56e1889edb4373b060739c876...,テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品,2500,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,37,0,0,1-2日後,...,2000-01-01,0,神奈川県,送料込,https://fril.jp/shop/d16cd0f0dd452bb98787cc349...,なし,https://img.fril.jp/img/152839505/l/513856974....,テレカ 未使用 50度数 レア 仮面ライダー 誕生20周年 レーザーディスク販売時の販促...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
1,2019/10/09 15:12:05,https://item.fril.jp/ac2ac2e9968e30e868e1cc87b...,送料無料 新品 DVD Perfume WORLD TOUR 1st 初回,4800,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,60,0,0,1-2日後,...,2000-01-01,0,京都府,送料込,https://fril.jp/shop/a1c14809b40f535a4ec33b213...,なし,https://img.fril.jp/img/154299175/l/439614345....,新品 DVD Perfume WORLD TOUR 1st 初回プレス盤 ご覧頂きあり...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
2,2019/10/09 15:12:05,https://item.fril.jp/797582a80b28b6c07bfe84c03...,☆ウルトラマン☆レーザーディスク ジャンク？,1980,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,87,0,1,1-2日後,...,2000-01-01,0,岡山県,送料込,https://fril.jp/shop/5d3613f833a4ee3a15e9dd72f...,なし,https://img.fril.jp/img/244479040/l/693281987....,ウルトラマンのレーザーディスクです 再生機器が無いのと 未開封のようですので現状渡しで...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
3,2019/10/09 15:12:05,https://item.fril.jp/5da5209c6bd09ceef11d1ed2d...,MAISHA （マイシャ）Sadao Watanabe 渡辺貞夫★LD,2600,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,9,0,0,1-2日後,...,2000-01-01,0,東京都,着払い,https://fril.jp/shop/22d352b7a529906589dcdbc60...,なし,https://img.fril.jp/img/124947216/l/353856650....,ご覧いただきありがとうございます MAISHA マイシャ 1985年 Sadao...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
4,2019/10/09 15:12:05,https://item.fril.jp/a06a93f70dce76664c0054b53...,LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA,3000,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,9,0,0,1-2日後,...,2000-01-01,0,東京都,着払い,https://fril.jp/shop/22d352b7a529906589dcdbc60...,なし,https://img.fril.jp/img/124947258/l/353856770....,ご覧いただきありがとうございます PARKER S MOOD パーカーズ ムード LI...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63001,2019/07/12 20:23:30,https://item.fril.jp/4a79945d051dec14200c85503...,64 マリオテニス,500,NINTENDO64,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,368,2,5,4~7日で発送,...,2000-01-01,0,熊本県,送料込,https://fril.jp/shop/35f887a74a9739c8d9e5d026f...,なし,https://img.fril.jp/img/220330456/l/626462275....,任天堂64のカセットです 裏にイニシャル記載あり 箱 説明書などあります 動作確認...,"{'カテゴリ': 'エンタメ/ホビー›テレビゲーム›家庭用ゲームソフト', 'サイズ': '...",エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
63002,2019/06/16 15:30:53,https://item.fril.jp/6bc3baba28c29a4681417ac56...,☆非売品☆switch ポケットモンスター ポケモンカードバトル クリアファイル,950,NintendoSwitch,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,372,1,4,3,...,2000-01-01,0,山口県,送料込,https://fril.jp/shop/5f35d399309625c1388e183a1...,なし,https://img.fril.jp/img/236315137/l/670680999....,新品未開封非売品ですソフトに付いてましたクリアファイル応募券付きです丁寧に雨対策をして台紙を...,"{'カテゴリ': 'エンタメ/ホビー›テレビゲーム›家庭用ゲームソフト', 'サイズ': '...",エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
63003,2019/06/13 01:12:35,https://item.fril.jp/a7c498013a4fc4ac43f8cc014...,【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）,8500,NintendoSwitch,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,95,1,2,2,...,2000-01-01,0,東京都,送料込,https://fril.jp/shop/23707e46a8542b94dadaeff32...,なし,https://img.fril.jp/img/224384559/l/637663265....,新品で未開封です大人気商品のため在庫残りわずかです特別にクロスクリーナーも付いています受注後...,"{'カテゴリ': 'エンタメ/ホビー›テレビゲーム›家庭用ゲームソフト', 'サイズ': '...",エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
63004,2019/06/13 02:52:50,https://item.fril.jp/a7c498013a4fc4ac43f8cc014...,【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）,8500,NintendoSwitch,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,95,1,2,2,...,2000-01-01,0,東京都,送料込,https://fril.jp/shop/23707e46a8542b94dadaeff32...,なし,https://img.fril.jp/img/224384559/l/637663265....,新品で未開封です大人気商品のため在庫残りわずかです特別にクロスクリーナーも付いています受注後...,"{'カテゴリ': 'エンタメ/ホビー›

In [21]:
data.category.nunique()

22

# テキストの前処理

In [106]:
data.title

0           テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品
1           送料無料 新品 DVD Perfume WORLD TOUR 1st 初回
2                          ☆ウルトラマン☆レーザーディスク ジャンク？
3             MAISHA （マイシャ）Sadao Watanabe　渡辺貞夫★LD
4        LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA
                           ...                   
63001                                   64 マリオテニス
63002    ☆非売品☆switch ポケットモンスター ポケモンカードバトル クリアファイル
63003    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
63004    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
63005    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
Name: title, Length: 63006, dtype: object

In [54]:
emoji.UNICODE_EMOJI

{'en': {'🥇': ':1st_place_medal:',
  '🥈': ':2nd_place_medal:',
  '🥉': ':3rd_place_medal:',
  '🆎': ':AB_button_(blood_type):',
  '🏧': ':ATM_sign:',
  '🅰': ':A_button_(blood_type):',
  '🇦🇫': ':Afghanistan:',
  '🇦🇱': ':Albania:',
  '🇩🇿': ':Algeria:',
  '🇦🇸': ':American_Samoa:',
  '🇦🇩': ':Andorra:',
  '🇦🇴': ':Angola:',
  '🇦🇮': ':Anguilla:',
  '🇦🇶': ':Antarctica:',
  '🇦🇬': ':Antigua_&_Barbuda:',
  '♒': ':Aquarius:',
  '🇦🇷': ':Argentina:',
  '♈': ':Aries:',
  '🇦🇲': ':Armenia:',
  '🇦🇼': ':Aruba:',
  '🇦🇨': ':Ascension_Island:',
  '🇦🇺': ':Australia:',
  '🇦🇹': ':Austria:',
  '🇦🇿': ':Azerbaijan:',
  '🔙': ':BACK_arrow:',
  '🅱': ':B_button_(blood_type):',
  '🇧🇸': ':Bahamas:',
  '🇧🇭': ':Bahrain:',
  '🇧🇩': ':Bangladesh:',
  '🇧🇧': ':Barbados:',
  '🇧🇾': ':Belarus:',
  '🇧🇪': ':Belgium:',
  '🇧🇿': ':Belize:',
  '🇧🇯': ':Benin:',
  '🇧🇲': ':Bermuda:',
  '🇧🇹': ':Bhutan:',
  '🇧🇴': ':Bolivia:',
  '🇧🇦': ':Bosnia_&_Herzegovina:',
  '🇧🇼': ':Botswana:',
  '🇧🇻': ':Bouvet_Island:',
  '🇧🇷': ':Brazil:',
  '🇮🇴': ':Britis

In [55]:
emoji.UNICODE_EMOJI['en'].keys()

dict_keys(['🥇', '🥈', '🥉', '🆎', '🏧', '🅰', '🇦🇫', '🇦🇱', '🇩🇿', '🇦🇸', '🇦🇩', '🇦🇴', '🇦🇮', '🇦🇶', '🇦🇬', '♒', '🇦🇷', '♈', '🇦🇲', '🇦🇼', '🇦🇨', '🇦🇺', '🇦🇹', '🇦🇿', '🔙', '🅱', '🇧🇸', '🇧🇭', '🇧🇩', '🇧🇧', '🇧🇾', '🇧🇪', '🇧🇿', '🇧🇯', '🇧🇲', '🇧🇹', '🇧🇴', '🇧🇦', '🇧🇼', '🇧🇻', '🇧🇷', '🇮🇴', '🇻🇬', '🇧🇳', '🇧🇬', '🇧🇫', '🇧🇮', '🆑', '🆒', '🇰🇭', '🇨🇲', '🇨🇦', '🇮🇨', '♋', '🇨🇻', '♑', '🇧🇶', '🇰🇾', '🇨🇫', '🇪🇦', '🇹🇩', '🇨🇱', '🇨🇳', '🇨🇽', '🎄', '🇨🇵', '🇨🇨', '🇨🇴', '🇰🇲', '🇨🇬', '🇨🇩', '🇨🇰', '🇨🇷', '🇭🇷', '🇨🇺', '🇨🇼', '🇨🇾', '🇨🇿', '🇨🇮', '🇩🇰', '🇩🇬', '🇩🇯', '🇩🇲', '🇩🇴', '🔚', '🇪🇨', '🇪🇬', '🇸🇻', '🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f', '🇬🇶', '🇪🇷', '🇪🇪', '🇸🇿', '🇪🇹', '🇪🇺', '🆓', '🇫🇰', '🇫🇴', '🇫🇯', '🇫🇮', '🇫🇷', '🇬🇫', '🇵🇫', '🇹🇫', '🇬🇦', '🇬🇲', '♊', '🇬🇪', '🇩🇪', '🇬🇭', '🇬🇮', '🇬🇷', '🇬🇱', '🇬🇩', '🇬🇵', '🇬🇺', '🇬🇹', '🇬🇬', '🇬🇳', '🇬🇼', '🇬🇾', '🇭🇹', '🇭🇲', '🇭🇳', '🇭🇰', '🇭🇺', '🆔', '🇮🇸', '🇮🇳', '🇮🇩', '🇮🇷', '🇮🇶', '🇮🇪', '🇮🇲', '🇮🇱', '🇮🇹', '🇯🇲', '🇯🇵', '🉑', '🈸', '🉐', '🏯', '㊗', '🈹', '🎎', '🈚', '🈁', '🈷', '🈵', '🈶', '🈺', '🈴', '🏣', '🈲', '🈯', '㊙', '🈂', '🔰', '🈳', '🇯🇪', '🇯🇴', '🇰🇿'

In [52]:
emojis = ''
for key in emoji.UNICODE_EMOJI.keys():
    emojis += "".join(emoji.UNICODE_EMOJI[key].keys())

In [53]:
emojis[:100]

'🥇🥈🥉🆎🏧🅰🇦🇫🇦🇱🇩🇿🇦🇸🇦🇩🇦🇴🇦🇮🇦🇶🇦🇬♒🇦🇷♈🇦🇲🇦🇼🇦🇨🇦🇺🇦🇹🇦🇿🔙🅱🇧🇸🇧🇭🇧🇩🇧🇧🇧🇾🇧🇪🇧🇿🇧🇯🇧🇲🇧🇹🇧🇴🇧🇦🇧🇼🇧🇻🇧🇷🇮🇴🇻🇬🇧🇳🇧🇬🇧🇫🇧🇮🆑🆒🇰🇭🇨🇲🇨🇦🇮🇨♋🇨🇻♑🇧🇶'

In [59]:
puncs = string.punctuation +  "◆▼★②●☆■★【】『』「」、♪"
puncs

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~◆▼★②●☆■★【】『』「」、♪'

In [63]:
#半角から全角
def han2zen(txt):
    txt = jctconv.h2z(txt, kana=True, digit=False, ascii=False)
    txt = jctconv.z2h(txt, kana=False, digit=True, ascii=True)
    return txt

def remove_signs(txt):
    rm_signs = emojis + puncs
    for sign in rm_signs:
        txt = txt.replace(sign, " ")
        
    return txt

def clean_txt(txt):
    txt = han2zen(txt)
    txt = remove_signs(txt)
    txt_list = txt.upper().split()
    txt_list = [x for x in txt_list if len(x) > 1 and re.search(r"[亜-熙ぁ-んァ-ヶa-zA-Z]", x)]
    return " ".join(list(Counter(txt_list)))

In [64]:
data.title

0           テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品
1           送料無料 新品 DVD Perfume WORLD TOUR 1st 初回
2                          ☆ウルトラマン☆レーザーディスク ジャンク？
3             MAISHA （マイシャ）Sadao Watanabe　渡辺貞夫★LD
4        LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA
                           ...                   
63001                                   64 マリオテニス
63002    ☆非売品☆switch ポケットモンスター ポケモンカードバトル クリアファイル
63003    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
63004    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
63005    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
Name: title, Length: 63006, dtype: object

In [66]:
data.title.head().apply(clean_txt)

0         テレカ 未使用品 仮面ライダー 周年 東映ビデオ販売 販促用非売品
1      送料無料 新品 DVD PERFUME WORLD TOUR ST 初回
2                      ウルトラマン レーザーディスク ジャンク
3        MAISHA マイシャ SADAO WATANABE 渡辺貞夫 LD
4    LD PARKER MOOD パーカーズ・ムード LIVE AT BRAVA
Name: title, dtype: object

In [65]:
data["clean_title"] = data.title.apply(clean_txt)

In [67]:
data["clean_title"].head()

0         テレカ 未使用品 仮面ライダー 周年 東映ビデオ販売 販促用非売品
1      送料無料 新品 DVD PERFUME WORLD TOUR ST 初回
2                      ウルトラマン レーザーディスク ジャンク
3        MAISHA マイシャ SADAO WATANABE 渡辺貞夫 LD
4    LD PARKER MOOD パーカーズ・ムード LIVE AT BRAVA
Name: clean_title, dtype: object

# テキストを数値に置き換える

In [68]:
# sklearn
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [69]:
category2idx = {c: idx for idx, c in enumerate(data.category.unique())}
category2idx

{'エンタメ/ホビー > エンタメその他 > その他': 0,
 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)': 1,
 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)': 2,
 'エンタメ/ホビー > CD > アニメ': 3,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ': 4,
 'エンタメ/ホビー > アニメグッズ > その他': 5,
 'エンタメ/ホビー > フィギュア > アニメ/ゲーム': 6,
 'エンタメ/ホビー > フィギュア > 特撮': 7,
 'レディース > ワンピース > ひざ丈ワンピース': 8,
 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー': 9,
 'スポーツ/アウトドア > フィッシング > ルアー用品': 10,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ': 11,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル': 12,
 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)': 13,
 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン': 14,
 'エンタメ/ホビー > トレーディングカード > シングルカード': 15,
 '楽器 > ギター > エレキギター': 16,
 'メンズ > 靴/シューズ > スニーカー': 17,
 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー': 18,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー': 19,
 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体': 20,
 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト': 21}

In [70]:
idx2category = {idx: c for idx, c in enumerate(data.category.unique())}
idx2category

{0: 'エンタメ/ホビー > エンタメその他 > その他',
 1: 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)',
 2: 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)',
 3: 'エンタメ/ホビー > CD > アニメ',
 4: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ',
 5: 'エンタメ/ホビー > アニメグッズ > その他',
 6: 'エンタメ/ホビー > フィギュア > アニメ/ゲーム',
 7: 'エンタメ/ホビー > フィギュア > 特撮',
 8: 'レディース > ワンピース > ひざ丈ワンピース',
 9: 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー',
 10: 'スポーツ/アウトドア > フィッシング > ルアー用品',
 11: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ',
 12: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル',
 13: 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)',
 14: 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン',
 15: 'エンタメ/ホビー > トレーディングカード > シングルカード',
 16: '楽器 > ギター > エレキギター',
 17: 'メンズ > 靴/シューズ > スニーカー',
 18: 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー',
 19: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー',
 20: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体',
 21: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト'}

In [71]:
data.category.apply(lambda x:category2idx[x])

0         0
1         0
2         0
3         0
4         0
         ..
63001    21
63002    21
63003    21
63004    21
63005    21
Name: category, Length: 63006, dtype: int64

In [109]:
X = data.clean_title
y = data.category.apply(lambda x:category2idx[x])

In [110]:
X

0              テレカ 未使用品 仮面ライダー 周年 東映ビデオ販売 販促用非売品
1           送料無料 新品 DVD PERFUME WORLD TOUR ST 初回
2                           ウルトラマン レーザーディスク ジャンク
3             MAISHA マイシャ SADAO WATANABE 渡辺貞夫 LD
4         LD PARKER MOOD パーカーズ・ムード LIVE AT BRAVA
                          ...                   
63001                                     マリオテニス
63002    非売品 SWITCH ポケットモンスター ポケモンカードバトル クリアファイル
63003     新品未開封 大乱闘スマッシュブラザーズ SPECIAL クロスクリーナー付き
63004     新品未開封 大乱闘スマッシュブラザーズ SPECIAL クロスクリーナー付き
63005     新品未開封 大乱闘スマッシュブラザーズ SPECIAL クロスクリーナー付き
Name: clean_title, Length: 63006, dtype: object

In [111]:
y

0         0
1         0
2         0
3         0
4         0
         ..
63001    21
63002    21
63003    21
63004    21
63005    21
Name: category, Length: 63006, dtype: int64

# データの学習

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [113]:
pipeline = Pipeline([
    ("bow", CountVectorizer()),
    ("tfidf", TfidfTransformer()),
    ("classifier", RandomForestClassifier())
])

In [114]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [115]:
pred = pipeline.predict(X_test)

In [116]:
pred

array([ 9,  1, 10, ..., 20, 17, 19])

In [117]:
pipeline2 = Pipeline([
    ("bow", CountVectorizer()),
    ("tfidf", TfidfTransformer()),
    ("classifier", MultinomialNB())
])

In [118]:
pipeline2.fit(X_train, y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [119]:
pred2 = pipeline2.predict(X_test)

In [120]:
pred2

array([ 9,  1, 10, ..., 20, 17, 19])

# 精度を評価する

In [121]:
# RandomForestClassifier
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.84      0.69      0.76       406
           1       0.89      0.93      0.91      1189
           2       0.87      0.77      0.82       422
           3       0.87      0.85      0.86       699
           4       0.76      0.83      0.80      3577
           5       0.71      0.38      0.50       395
           6       0.89      0.83      0.86      2138
           7       0.85      0.83      0.84      1033
           8       0.93      0.89      0.91       460
           9       0.93      0.82      0.87       379
          10       0.97      0.86      0.91       871
          11       0.81      0.58      0.67       481
          12       0.87      0.76      0.81       430
          13       0.94      0.82      0.87       399
          14       0.99      0.96      0.97       535
          15       0.95      0.91      0.93       830
          16       0.98      0.82      0.90       396
          17       0.98    

In [122]:
# MultinomialNB
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

           0       0.95      0.19      0.32       406
           1       0.82      0.90      0.86      1189
           2       0.93      0.50      0.65       422
           3       0.98      0.51      0.67       699
           4       0.55      0.91      0.68      3577
           5       1.00      0.04      0.07       395
           6       0.77      0.87      0.82      2138
           7       0.90      0.63      0.74      1033
           8       1.00      0.60      0.75       460
           9       0.99      0.59      0.74       379
          10       0.99      0.75      0.85       871
          11       1.00      0.01      0.02       481
          12       0.99      0.43      0.61       430
          13       0.99      0.34      0.50       399
          14       1.00      0.91      0.95       535
          15       0.99      0.84      0.91       830
          16       1.00      0.61      0.75       396
          17       0.77    

In [123]:
idx2category

{0: 'エンタメ/ホビー > エンタメその他 > その他',
 1: 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)',
 2: 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)',
 3: 'エンタメ/ホビー > CD > アニメ',
 4: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ',
 5: 'エンタメ/ホビー > アニメグッズ > その他',
 6: 'エンタメ/ホビー > フィギュア > アニメ/ゲーム',
 7: 'エンタメ/ホビー > フィギュア > 特撮',
 8: 'レディース > ワンピース > ひざ丈ワンピース',
 9: 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー',
 10: 'スポーツ/アウトドア > フィッシング > ルアー用品',
 11: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ',
 12: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル',
 13: 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)',
 14: 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン',
 15: 'エンタメ/ホビー > トレーディングカード > シングルカード',
 16: '楽器 > ギター > エレキギター',
 17: 'メンズ > 靴/シューズ > スニーカー',
 18: 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー',
 19: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー',
 20: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体',
 21: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト'}

# 学習済みモデルの保存

In [124]:
# target = ["NIKE AIR FORCE 1 LOW QS PUERTO RICO 2020"]
target = ["ニューバランス Cross Country 25.5cm M320"]
target = ["✨新品✨オニツカタイガー ❇️25.0cm"]
target = ["ビームス　ナイキ　REACT PRESTO DHARMA"]
prediction = pipeline.predict(target)

prediction

array([17])

In [125]:
idx2category[prediction[0]]

'メンズ > 靴/シューズ > スニーカー'

In [126]:
import pickle

In [127]:
with open("rdmf.pickle", mode="wb") as f:
    pickle.dump(pipeline, f)

In [129]:
#読み込むときは
with open("rdmf.pickle", mode="rb") as ff:
    model = pickle.load(ff)

In [130]:
model.predict(target)

array([17])

## カテゴリーも保存しておく

In [131]:
idx2category

{0: 'エンタメ/ホビー > エンタメその他 > その他',
 1: 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)',
 2: 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)',
 3: 'エンタメ/ホビー > CD > アニメ',
 4: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ',
 5: 'エンタメ/ホビー > アニメグッズ > その他',
 6: 'エンタメ/ホビー > フィギュア > アニメ/ゲーム',
 7: 'エンタメ/ホビー > フィギュア > 特撮',
 8: 'レディース > ワンピース > ひざ丈ワンピース',
 9: 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー',
 10: 'スポーツ/アウトドア > フィッシング > ルアー用品',
 11: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ',
 12: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル',
 13: 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)',
 14: 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン',
 15: 'エンタメ/ホビー > トレーディングカード > シングルカード',
 16: '楽器 > ギター > エレキギター',
 17: 'メンズ > 靴/シューズ > スニーカー',
 18: 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー',
 19: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー',
 20: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体',
 21: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト'}

In [132]:
pd.DataFrame([
    {"k": k, "v": v}
    for k, v in idx2category.items()
]).to_csv("idx2category.csv", index=False)